In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
nltk.download('stopwords')
nltk.download('punkt')

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /home/varun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/varun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
dataframe = pd.read_csv('Sentiment.csv')
dataframe = dataframe[dataframe.sentiment != 'Neutral']

In [3]:
y = dataframe['sentiment'].values
x = dataframe['text'].values

In [4]:
type(y)

numpy.ndarray

In [5]:
for i in range(len(y)):
    if y[i] == 'Negative':
        y[i] = 0
    else:
        y[i] = 1

In [6]:
stop = list(string.punctuation)

In [7]:
for i in range(len(x)):
    x[i] = [j for j in word_tokenize(x[i].lower()) if j not in stop]

In [8]:
type(x)
len(max(x,key=len))

34

In [9]:
from gensim import corpora
dictionary = corpora.Dictionary(x)
dictionary.save('/tmp/twitter.dict')
# print(dictionary.token2id)

In [10]:
len(dictionary.token2id)

16051

In [11]:
for i in range(len(x)):
    for j in range(len(x[i])):
        x[i][j] = dictionary.token2id[x[i][j]] 

In [12]:
x

array([list([17, 19, 6, 14, 5, 22, 7, 8, 12, 15, 9, 3, 21, 16, 18, 0, 4, 13, 11, 2, 20, 23, 10, 1]),
       list([17, 37, 40, 28, 30, 33, 41, 24, 32, 26, 31, 29, 24, 25, 27, 16, 22, 35, 11, 34, 8, 38, 39, 0, 36, 42]),
       list([17, 44, 8, 53, 51, 46, 22, 47, 50, 11, 22, 48, 16, 49, 45, 52, 10, 43]),
       ...,
       list([17, 14117, 71, 136, 376, 65, 72, 1510, 1215, 22, 213, 74, 546, 717, 65, 478, 65, 346, 546, 7070]),
       list([17, 737, 8, 536, 226, 38, 40, 311, 196, 14, 278, 179, 150, 154, 387, 133, 196, 444, 169, 55, 436, 738, 54, 739, 515, 42]),
       list([17, 14117, 71, 14308, 2032, 22, 49, 45, 817, 317, 7070, 10, 14307])],
      dtype=object)

In [13]:
max_words = 36
from keras.preprocessing import sequence
X_train = sequence.pad_sequences(x, maxlen=max_words)

In [14]:
X_train.shape

(10729, 36)

In [15]:
X_train[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17, 19,  6, 14,  5,
       22,  7,  8, 12, 15,  9,  3, 21, 16, 18,  0,  4, 13, 11,  2, 20, 23,
       10,  1], dtype=int32)

In [16]:
embedding_size=32       #Maintain
model=Sequential()
model.add(Embedding(16051,embedding_size, input_length=36))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 36, 32)            513632    
_________________________________________________________________
lstm_1 (LSTM)                (None, 36, 100)           53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 597,083
Trainable params: 597,083
Non-trainable params: 0
_________________________________________________________________
None

In [17]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [18]:
batch_size = 64
num_epochs = 3
model.fit(X_train, y, batch_size=batch_size, epochs=num_epochs)

Epoch 1/3
10729/10729 [==============================] - 10s 887us/step - loss: 0.4362 - acc: 0.8191
Epoch 2/3
10729/10729 [==============================] - 10s 962us/step - loss: 0.2649 - acc: 0.8914
Epoch 3/3
10729/10729 [==============================] - 12s 1ms/step - loss: 0.1820 - acc: 0.9290


In [19]:
sentence = [['i','am','so','happy']]
for i in range(len(sentence)):
    for j in range(len(sentence[i])):
        sentence[i][j] = dictionary.token2id[sentence[i][j]]   


In [20]:
testArray = sequence.pad_sequences(sentence, maxlen=36)

In [21]:
testArray

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,  65, 745, 281,  78]], dtype=int32)

In [22]:
model.predict(testArray)

array([[0.30202144]], dtype=float32)